### Procesamiento **ETL** al archivo **`output_steam_games.json`**

In [1]:
# Importamos las librerias necesarias para el procesamiento de los datos
import pandas as pd
import numpy as np
import json
import ast

In [2]:
filas = list()
with open("data/australian_users_items.json", "r", encoding="Latin-1") as archivo:
    for linea in archivo.readlines():
        filas.append(ast.literal_eval(linea))

df_users_items = pd.DataFrame(filas)
df_users_items

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...,...
88305,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,[]
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"[{'item_id': '304930', 'item_name': 'Unturned'..."


In [3]:
df_users_items.info()  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB


### Limpieza de la columna **``items``**
Este código primero **`explota`** la columna **`items`**, lo que significa que crea una nueva fila para cada elemento en las listas de la columna **`items`**. Luego, utiliza **`json_normalize()`** para desanidar los objetos **`JSON`** en la columna **`items`**. Finalmente, elimina la columna **`items`** del DataFrame original y une el DataFrame original con el DataFrame desanidado.

In [4]:
df_items_clean = pd.json_normalize(df_users_items['items'].explode())
df_users_items = df_items_clean.join(df_users_items.drop('items', axis=1))

In [5]:
df_users_items.head(2)  

,item_id,item_name,playtime_forever,playtime_2weeks,user_id,items_count,steam_id,user_url
0,10,Counter-Strike,6.0,0.0,76561197970982479,277.0,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,20,Team Fortress Classic,0.0,0.0,js41637,888.0,76561198035864385,http://steamcommunity.com/id/js41637


In [6]:
df_users_items.shape

(5170015, 8)

In [7]:
# contar nulos
df_users_items.isnull().sum()

item_id               16806
item_name             16806
playtime_forever      16806
playtime_2weeks       16806
user_id             5081705
items_count         5081705
steam_id            5081705
user_url            5081705
dtype: int64

In [10]:
df_users_items.count()

item_id             5153209
item_name           5153209
playtime_forever    5153209
playtime_2weeks     5153209
user_id               88310
items_count           88310
steam_id              88310
user_url              88310
dtype: int64

Eliminamos los nulos y los datos donde **``user_id ``** es nulo e **``items_count``** significa que no posee juegos

In [25]:
df_users_items = df_users_items.dropna(how='all', subset=['user_id', 'items_count', 'items_count'])
# Eliminar los usuarios que no poseen juegos (items_count igual a 0)
df_users_items = df_users_items[df_users_items['items_count'] > 0]
# Contamos los valores nulos nuevamente 
df_users_items.isna().sum()

item_id             112
item_name           112
playtime_forever    112
playtime_2weeks     112
user_id               0
items_count           0
steam_id              0
user_url              0
dtype: int64

In [26]:
df_users_items.dropna(inplace=True)

Eliminamos las columnas que no usaremos 

In [27]:
# Procedemos a eliminar las columnas que no son relevantes para nuestro futuro modelo
df_users_items.drop(columns=["user_url","steam_id","playtime_2weeks"], inplace=True)

### Limpieza de la columna **``playtime_forever``** 

Realizamos unas transformaciones necesarias al **`DataFrame`**:

1. **Convertimos de minutos a horas** creamos una función llamada **`transform`** toma el DataFrame **`df_users_items`** como argumento y divide la columna **`playtime_forever`** por **`60`**. Esto convierte el tiempo de juego, que originalmente está en minutos, a horas. Luego, esta función devuelve el DataFrame modificado.


2. **Hacemos una conversión de tipos de datos** convertimos los tipos de datos de las columnas **``item_id``** y **`item_name`**. La columna **`item_id`** se convierte a **`int`** (entero), y la columna **`item_name`** se convierte a string (cadena de texto). Esto se hace para asegurar que estos campos tengan los tipos de datos correctos para el análisis y el modelado futuro.

3. **Eliminamos los usuarios sin tiempo de juego** Se eliminan los usuarios que no tienen tiempo de juego registrado. Esto se hace porque estos usuarios no aportan información relevante para el análisis y modelado de los datos.

In [28]:
def transform (df_items):
    # Conversión de 'playtime_forever' a horas
    df_items['playtime_forever'] /= 60

    # Conversión de tipo de dato para 'item_id'
    df_items['item_id'] = df_items['item_id'].astype(int)

    # Conversión de tipo de dato para 'item_name'
    df_items['item_name'] = df_items['item_name'].astype('string')

    # Eliminar usuarios sin tiempo de juego
    df_items.drop(df_items[df_items['playtime_forever'] == 0].index, inplace=True)

# Aplicar la función al DataFrame
transform (df_users_items)

df_users_items.info()

<class 'pandas.core.frame.DataFrame'>
Index: 48328 entries, 0 to 88308
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   item_id           48328 non-null  int32  
 1   item_name         48328 non-null  string 
 2   playtime_forever  48328 non-null  float64
 3   user_id           48328 non-null  object 
 4   items_count       48328 non-null  float64
dtypes: float64(2), int32(1), object(1), string(1)
memory usage: 2.0+ MB


In [29]:
df_users_items = df_users_items.reset_index(drop=True)

In [30]:
df_users_items

,item_id,item_name,playtime_forever,user_id,items_count
0,10,Counter-Strike,0.100000,76561197970982479,277.0
1,30,Day of Defeat,0.116667,evcentric,137.0
2,300,Day of Defeat: Source,78.883333,maplemage,629.0
3,3830,Psychonauts,5.550000,corrupted_soul,115.0
4,3900,Sid Meier's Civilization IV,5.633333,jorellpogi,71.0
...,...,...,...,...,...
48323,213650,Dwarfs F2P,1.033333,76561198320136420,321.0
48324,219340,The Banner Saga: Factions,0.183333,ArkPlays7,4.0
48325,202090,Magicka: Wizard Wars,25.016667,76561198323066619,22.0
48326,239220,The Mighty Quest For Epic Loot,39.566667,76561198326700687,177.0


Convertimos el **`DataFrame`** **`df_users_items`** a un archivo **`Parquet`** utilizando el método **`to_parquet()`**

In [31]:
df_users_items.to_parquet('items.parquet')